In [153]:
import numpy as np
import matplotlib.pyplot as plt



In [154]:
def diff(vect, out):
    shape = vect.shape
    N = shape[0]
    for j in range(N+1):
        if j == 0:
            out[j] = vect[j]
        elif (j>0) & (j <= N-1):
            out[j] = vect[j]-vect[j-1]
        else:
            out[j] = -vect[N-1]


def dave(vect, out):
    N = len(vect)
    for j in range(N+1):
        if j == 0:
            out[j] = vect[j]/2
        elif (j>0) & (j <= N-1):
            out[j] = (vect[j]+vect[j-1])/2
        else:
            out[j] = vect[N-1]/2


def skew(axis):
    U = np.array([[0,-axis[2],axis[1]],[axis[2],0,-axis[0]],[-axis[1],axis[0],0]])
    return U

def rodrigues(vector):
    #Vector is quantity dt/2*wi_L
    theta = np.linalg.norm(vector)
    #print(theta)
    if np.linalg.norm(vector) != 0:
        axis = vector/np.linalg.norm(vector)
    else:
        axis = vector
    #print(axis)
    U = skew(axis)
    R = np.identity(3)+np.sin(theta)*U+(1-np.cos(theta))*U@U    
    return R

def logMatrix(matrix):
    # log(R) = R - R.T  Skew symm matrix, must convert back to vector
    theta = np.arccos((np.trace(matrix)-1)/2)
    if theta == 0:
        return np.zeros(3)
    temp = matrix - matrix.T
    output = np.zeros(3)
    output[0] = temp[2][1]
    output[1] = temp[0][2]
    output[2] = temp[1][0]
    output = output/np.linalg.norm(output)
    return output*theta


def d_from_Q(d,Q_list):
    d = []
    for i in range(len(Q_list)):
        directors = []
        for j in range(3):
            directors.append(Q_list[i][j])
        d.append(directors)
    return d

def l_from_r(r_list):
    #temp = np.pad(r_list,(1,0), 'constant', constant_values = (0,0))
    #temp2 = np.pad(r_list,(0,1), 'constant', constant_values = (0,0))
    
    temp = np.vstack((np.array([0,0,0]),r_list))
    temp2 = np.vstack((r_list,np.array([0,0,0])))
    temp3 = temp2-temp
    return(temp3[1:len(r_list)])
    
def t_from_l(t_array,l_array):
    for i in range(len(l_array)):
        t_array[i]=l_array[i]/np.linalg.norm(l_array[i])
    
def e_from_l(e_array,l_array):
    for i in range(len(l_array)):
        e_array[i]=np.linalg.norm(l_array[i])/chunk_length


In [155]:
'''Beam Validation'''

rho = 5*10**3      #density  (kg/m^3)
E = 1*10**6        #Young's modulus (Pa)
G = 1*10**4        #shear modulus (Pa)
F_mag = 15     #force (N)
L = 3              #rod length (m)
RADIUS = 0.25      #rod radius (m)

T = 5*10**3        #simulation time (s)
n = 10            #number of discrete pts 
gamma = 500/n       #dissipation const (kg/(m*s))
dt = 3*10**(-4)    #time step (s)
#e = 1             #stretch factor

chunk_length = L/(n-1)

AREA = np.pi * RADIUS**2
MASS = AREA * chunk_length * rho

I1 = np.pi*RADIUS**4/4
I2 = np.pi*RADIUS**4/4
I3 = np.pi*RADIUS**4/2

### Stiffness Matricies, Constant
J = np.diag(np.array([MASS*chunk_length**2/12,MASS*chunk_length**2/12,MASS*RADIUS**2/2]))# M l^2 / 12 , M l^2 / 12,  M*r^2 / 2,
S = np.diag(np.array([4*G*AREA/3, 4*G*AREA/3, E*AREA]))
B = np.diag(np.array([E*I1, E*I2, G*I3]))

#initialize rod
r = np.zeros((n,3))             #array of node coordinate vectors
v = np.zeros((n,3))             #array of node velocity vectors
Q = np.zeros((n-1,3,3))         #array of Q matrices for each element
w = np.zeros((n-1,3))           #array of angular velocity vectors for each element
F = np.zeros((n,3))             #array of Force vectors
#F[0] = np.array([-F_mag,0,0])
F[n-1] = np.array([0,0,-F_mag]) 
C = np.zeros((n-1,3))
sigma = np.zeros((n-1,3))         #shear vector

for i in range(n):
    r[i] = np.array([i*chunk_length,0,0])
#r[0] = [np.sqrt(3)/2*chunk_length]

matrix = np.zeros((3,3))
matrix[2] = np.array([1,0,0])
matrix[0] = np.array([0,0,1])
matrix[1] = np.cross(matrix[2],matrix[0])

for i in range(n-1):
    Q[i] = matrix  

l = l_from_r(r) #list containing tangent vectors
t = np.zeros((n-1,3))
e = np.zeros((n-1,1))
t_from_l(t,l) 
e_from_l(e,l)

for i in range(len(sigma)):
    sigma[i] = Q[i]@(e[i]*t[i]-d[i][2])
    
K = np.zeros((n-2,3))
beta = np.zeros((n-2,3,3))

for i in range(n-2):
    K[i] = logMatrix(Q[i+1]@Q[i].T)/chunk_length
    beta[i] = B

end_node = []



#fixedMidpt = np.copy(r[n//2])

#Time March
steps = 50000
for step in range(steps):
    #First Half Position Update
    #print('Step: ')
    #print(step)
    
    r = r + (dt/2)*v   #r half update
    r[0] = np.array([0,0,0])
    #r[n//2] = fixedMidpt
    
    
    for i in range(len(Q)):
        Q[i] = rodrigues(dt/2*w[i])@Q[i]
    Q[0] = matrix

    
    l = l_from_r(r)    #l half update
    t_from_l(t,l)    #t half update  
    e_from_l(e,l)    #e half update


    for i in range(len(sigma)):   #sigma half update
        sigma[i] = Q[i]@(e[i]*t[i]-Q[i][2])
    for i in range(n-2):
        K[i] = logMatrix(Q[i+1]@Q[i].T)/chunk_length
    
    #Local Accelerations
    QUANTITY = np.zeros((n-1,3))
    for i in range(n-1):
        QUANTITY[i] = Q[i].T@S@sigma[i]
    dv = diff(QUANTITY)+F
    dv = dv/MASS
    v = v + dt*dv
    
    bkap = np.zeros((n-2,3))
    bkross = np.zeros((n-2,3))
    for i in range(n-2):
        bkap[i] = beta[i]@K[i]
        bkross[i] = np.cross(K[i],bkap[i])*chunk_length
    QtcrossSsig = np.zeros((n-1,3))
    for i in range(n-1):
        QtcrossSsig[i] = np.cross(Q[i]@t[i],S@sigma[i])
    
    dw = diff(bkap) + dave(bkross) - QtcrossSsig*chunk_length + C
    for i in range(n-1):
        dw[i] = np.linalg.inv(J)@dw[i]
    w = w + dt*dw
    
    #Second Half Position Update
    r = r + dt/2*v
    r[0] = np.array([0,0,0])
    #r[n//2] = fixedMidpt
    
    for i in range(len(Q)):
        Q[i] = rodrigues(dt/2*w[i])@Q[i]
    Q[0] = matrix
    for i in range(len(F)):
        F[i] -= dv[i]*dt*gamma
    end_node.append(r[n-1][2])




TypeError: diff() missing 1 required positional argument: 'out'

In [ ]:
x = []
z = []
for pos in r:
    x.append(pos[0])
    z.append(pos[2])
plt.figure()    
plt.plot(x,z)
plt.figure()
plt.plot(end_node)

####
#### TEST FOR KINEMATICS, Butterfly type setup, set middle point to boundary condition
#### only a few links, add forces on either ends with no damping
####